# Neural Machine Translation
While it might not seem like it, one of the very first development in Artificial Intellience is Neural Machine Translation. Traditionally, machine translation is a challenging task that involves large statistical models developed using high sophisticated linguistic knowledge. In neural machine translation, deep neural networks are developed for the problem. Advancing toward using Artificial Intelligence in machine translation task, AI to look for patterns in the input language and provide the target language representations as output. 

## Data Preparation
Like many machine learning task, we need to start with the data. In this tutorial, we'll use a dataset of English to Vietnamese phrases. Think of this as learning Vietnamese or English using flashcards. The dataset can be download [here](https://www.kaggle.com/datasets/hungnm/englishvietnamese-translation). To prepare the dataset for modeling, we'll perform the following:

1. Start by reading in the associated data and scan through it
2. Cleanup punctuation
3. Process upper and lowercase words
4. Processing special characters
5. Handle duplciate phrases in English with different translations in Vietnamese

In [1]:
# import python libraries
import re
import numpy as np
from unicodedata import normalize

In [ ]:
## Helper Functions
def load_data(file_path);
    with open(file_path, 'r') as file:
        data = file.read()
    return data